In [1]:
import numpy as np
import mlgrad.model as model
import mlgrad.loss as loss
import mlgrad.func as func
import mlgrad.avragg as avragg
import mlgrad.gd as gd
import mlgrad.regular as regular
import mlgrad.weights as weights
from mlgrad.utils import array_exclude_outliers

import mlgrad.linreg as linreg

from mlgrad import averager_it, averager_fg, fg, erm_fg, sg, erm_sg, irgd, erm_irgd, erisk, mrisk

#import sklearn.linear_model as sklm

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use(['seaborn-notebook', 'seaborn-ticks'])

import sys
print(sys.version)

ModuleNotFoundError: No module named 'mlgrad.linreg'

In [ ]:
df = pd.read_csv("data/starsCYG.csv", sep=',', usecols=["log.Te", "log.light"], )

In [ ]:
len(df)

In [ ]:
Y = df["log.light"].values
X = df["log.Te"].values

X -= np.median(X)
Y -= np.median(Y)

# x_min, x_max = X.min(), X.max()
# X = (X - x_min) / (x_max - x_min)
# y_min, y_max = Y.min(), Y.max()
# Y = (Y - y_min) / (y_max - y_min)

Xs = X.reshape((-1,1))

In [ ]:
plt.figure(figsize=(6.0, 5.0))
plt.scatter(X, Y, s=64, c='w', edgecolors='k', linewidths=2.)
plt.minorticks_on()
plt.grid(1)
plt.xlabel('log.Te')
plt.ylabel('log.light')
# plt.xlim(-0.9, 0.6)
# plt.ylim(-2.5, 2.0)
plt.show()

In [ ]:
loss_func_sq = loss.ErrorLoss(func.Square())
loss_func_rsq = loss.RelativeErrorLoss(func.Square())
loss_func_abs = loss.ErrorLoss(func.Absolute())

In [ ]:
mod1 = model.LinearModel(1)
mod1.init()

print(np.array(mod1.param))

In [ ]:
# def regression(Xs, Y, mod, lossfunc=None, h=0.001, tol=1.0e-8, n_iter=1000, verbose=1, n_restart=5):
#     _lossfunc = lossfunc
#     if lossfunc is None:
#         _lossfunc = loss.SquareErrorLoss()
#     er = erisk(Xs, Y, mod, _lossfunc)
#     alg_fg = erm_fg(er, h=h, tol=tol, n_iter=n_iter, verbose=verbose, n_restart=5)
#     return alg_fg

In [ ]:
# er1 = erisk(Xs, Y, mod1, loss_func_sq)
# alg_fg1 = erm_fg(er1, h=0.001, tol=1.0e-8, verbose=1, n_restart=5)

alg_fg1 = linreg.regression(Xs, Y, mod1)

plt.plot(alg_fg1.lvals)
plt.minorticks_on()
plt.grid(1)
plt.show()

In [ ]:
def averaging_function(kind, **kw):
    if kind == 'M':
        alpha = kw.get('alpha', 0.5)
        func = kw.get('func', Quantile_Sqrt(alpha, 0.001))
        avg = averager_it(func)
    elif kind == 'WM':
        alpha = kw.get('alpha', 0.5)
        func = kw.get('func', Quantile_Sqrt(alpha, 0.001))
        avg = averager_it(func)
        avg = avragg.WMAverage(avg)
    else:
        raise ValueError('Invalid argument value')
    return avg
        

In [ ]:
# def m_regression(Xs, Y, mod, 
#                  lossfunc=None, avrfunc=None, h=0.001, 
#                  tol=1.0e-8, n_iter=1000, tol2=1.0e-5, n_iter2=21):
#     _lossfunc = lossfunc
#     if lossfunc is None:
#         _lossfunc = loss.SquareErrorLoss()

#     er = erisk(Xs, Y, mod, _lossfunc)
#     alg_fg = fg(er, h=h, tol=tol)

#     _avrfunc = avrfunc
#     if avrfunc is None:
#         _avrfunc = averager_it(func.Quantile_Sqrt(0.5, 0.001))
#         _avrfunc = avragg.WMAverage(_avrfunc)

#     wg = weights.MWeights(_avrfunc, er)
#     irgd = erm_irgd(alg_fg, wg, n_iter=n_iter2, tol=tol2)
        
#     return irgd

In [ ]:
mod2 = model.LinearModel(1)
mod2.init()

In [ ]:
# er2 = erisk(Xs, Y, mod2, loss_func_sq)
# alg_fg2 = fg(er2, h=0.001, tol=1.0e-9)
# avg2 = averager_it(func.Quantile_Sqrt(0.83, 0.001))
# avg2 = avragg.WMAverage(avg2)
# wg2 = weights.MWeights(avg2, er2)
# irgd2 = erm_irgd(alg_fg2, wg2, n_iter=21, tol=1.0e-5)

avg2 = averager_it(func.Quantile_Sqrt(0.84, 0.001))
avg2 = avragg.WMAverage(avg2)
irgd2 = linreg.m_regression(Xs, Y, mod2)

plt.plot(irgd2.lvals)
plt.minorticks_on()
plt.grid(1)
plt.show()

In [ ]:
res1 = np.abs(Y - mod1.evaluate_all(Xs))
res1.sort()
res2 = np.abs(Y - mod2.evaluate_all(Xs))
res2.sort()
#ranges = np.arange(len(res1))

plt.plot(res1, marker='o', label='ols')
plt.plot(res2, marker='o', label='rob')
plt.legend()
plt.show()

In [ ]:
Err = np.abs(Y - mod2.evaluate_all(Xs))

X1, X1_ = array_exclude_outliers(X, Err, 7) 
Y1, Y1_ = array_exclude_outliers(Y, Err, 7) 
X1s = X1.reshape(-1,1)

In [ ]:
plt.figure(figsize=(6.0, 5.0))
plt.title('StarsCYG dataset')
plt.scatter(X1, Y1, s=64, c='w', edgecolors='k', linewidths=1., label='regular data')
plt.scatter(X1_, Y1_, s=64, c='Grey', edgecolors='k', linewidths=1., label='outliers')
X0 = np.linspace(X.min(), X.max(), 20)
plt.plot(X0, mod1.evaluate_all(X0.reshape(-1,1)), c='k', linestyle='--', linewidth=2.5, label=r'LS')
plt.plot(X0, mod2.evaluate_all(X0.reshape(-1,1)), color='k', linewidth=2., label=r'WM')
plt.minorticks_on()
plt.grid(1)
plt.xlabel('log.Te')
plt.ylabel('log.light')
# plt.xlim(-0.9, 0.6)
# plt.ylim(-2.5, 2.0)
plt.legend(loc='best')
plt.show()

In [ ]:
mod21 = model.LinearModel(1)
mod21.init()

In [ ]:
er21 = erisk(Xs, Y, mod21, loss_func_sq)
alg_fg21 = fg(er21, h=0.001, tol=1.0e-8)
wg21 = weights.RWeights(func.Sqrt(1.0), er21)
irgd21 = erm_irgd(alg_fg21, wg21, n_iter=21, tol=1.0e-6)

plt.plot(irgd21.lvals)
plt.minorticks_on()
plt.grid(1)
plt.show()

In [ ]:
plt.figure(figsize=(6.0, 5.0))
plt.title('StarsCYG dataset')
plt.scatter(X1, Y1, s=64, c='w', edgecolors='k', linewidths=1., label='regular data')
plt.scatter(X1_, Y1_, s=64, c='Grey', edgecolors='k', linewidths=1., label='outliers')
X0 = np.linspace(-0.9, 0.6, 20)
plt.plot(X0, mod1.evaluate_all(X0.reshape(-1,1)), c='k', linestyle='--', linewidth=2.5, label=r'LS')
plt.plot(X0, mod21.evaluate_all(X0.reshape(-1,1)), color='k', linewidth=2., label=r'R')
plt.minorticks_on()
plt.grid(1)
plt.xlabel('log.Te')
plt.ylabel('log.light')
plt.xlim(-0.9, 0.6)
plt.ylim(-2.5, 2.0)
plt.legend(loc='best')
plt.show()

In [ ]:
mod22 = model.LinearModel(1)
mod22.init()

In [ ]:
avg22 = averager_it(func.Quantile_Sqrt(0.84, 0.001))
avg22 = avragg.WMAverage(avg22)
mr2 = mrisk(Xs, Y, mod22, loss_func_sq, avg22)
alg_fg2 = fg(mr2, h=0.01, tol=1.0e-7)
alg_fg2.fit()

plt.plot(alg_fg2.lvals)
plt.minorticks_on()
plt.grid(1)
plt.show()

In [ ]:
plt.figure(figsize=(6.0, 5.0))
plt.title('StarsCYG dataset')
plt.scatter(X1, Y1, s=64, c='w', edgecolors='k', linewidths=1., label='regular data')
plt.scatter(X1_, Y1_, s=64, c='Grey', edgecolors='k', linewidths=1., label='outliers')
X0 = np.linspace(-0.9, 0.6, 20)
plt.plot(X0, mod1.evaluate_all(X0.reshape(-1,1)), c='k', linestyle='--', linewidth=2.5, label=r'LS')
plt.plot(X0, mod22.evaluate_all(X0.reshape(-1,1)), color='k', linewidth=2., label=r'WM')
plt.minorticks_on()
plt.grid(1)
plt.xlabel('log.Te')
plt.ylabel('log.light')
plt.xlim(-0.9, 0.6)
plt.ylim(-2.5, 2.0)
plt.legend(loc='best')
plt.show()

In [ ]:
mod3 = model.LinearModel(1)
mod3.init()

In [ ]:
er3 = erisk(X1s, Y1, mod3, loss_func_sq)
alg_fg3 = erm_fg(er3, h=0.01, tol=1.0e-6, verbose=0)

plt.plot(alg_fg3.lvals)
plt.minorticks_on()
plt.grid(1)
plt.show()

In [ ]:
mod4 = model.LinearModel(1)
mod4.init()

In [ ]:
er4 = erisk(Xs, Y, mod4, loss_func_rsq)
alg_fg4 = fg(er4, h=0.001, tol=1.0e-9)
avg4 = averager_it(func.QuantileFunc(0.83, func.Sqrt(0.001)))
avg4 = avragg.HMAverage(avg4)
wg4 = weights.MWeights(avg4, er4)
irgd4 = erm_irgd(alg_fg4, wg4, n_iter=22, tol=1.0e-5)

plt.plot(irgd4.lvals)
plt.minorticks_on()
plt.grid(1)
plt.show()

In [ ]:
Err = np.abs(Y - mod4.evaluate_all(Xs))

X2, X2_ = array_exclude_outliers(X, Err, 6) 
Y2, Y2_ = array_exclude_outliers(Y, Err, 6) 
X2s = X2.reshape(-1,1)

In [ ]:
plt.figure(figsize=(6.0, 5.0))
plt.title('StarsCYG dataset')
plt.scatter(X2, Y2, s=64, c='w', edgecolors='k', linewidths=1., label='regular data')
plt.scatter(X2_, Y2_, s=64, c='Grey', edgecolors='k', linewidths=1., label='outliers')
X0 = np.linspace(-0.9, 0.6, 20)
plt.plot(X0, mod3.evaluate_all(X0.reshape(-1,1)), c='k', linestyle='--', linewidth=2.5, label=r'LS')
plt.plot(X0, mod4.evaluate_all(X0.reshape(-1,1)), color='k', linewidth=2., label=r'WM')
plt.plot(X0, mod2.evaluate_all(X0.reshape(-1,1)), c='k', linestyle=':', linewidth=2.5, label=r'HM')
plt.minorticks_on()
plt.grid(1)
plt.xlabel('log.Te')
plt.ylabel('log.light')
plt.legend(loc='best')
plt.xlim(-0.9, 0.6)
plt.ylim(-2.5, 2.0)
plt.show()

In [ ]:
res3 = np.abs(Y1 - mod2.evaluate_all(X1s))
res3.sort()
res4 = np.abs(Y1 - mod4.evaluate_all(X1s))
res4.sort()
plt.plot(res3, marker='s', label='HM')
plt.plot(res4, marker='s', label='WM')
plt.legend()
plt.show()

In [ ]:
40/47